In [1]:
from google.colab import auth
auth.authenticate_user()

In [2]:
!pip install geemap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 KB 25.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 KB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 68.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 KB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.6/99.6 KB 12.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 KB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 60.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [3]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=n7v5k3A1R1jKL1Wpya0NZd33kym2pAI75-Nd5J7zR0U&tc=fnTOBQbYTnTAKfEeO3oX8G3P6h87ihY0TAtRwoC3tfg&cc=QNMhr5m6pJbTn-vmcqw6uB-PJoyRNr55eizdFGXFM-k

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AWtgzh4xzgZ5XK30dNTMyG6usTvZJ7vf_4rNzp7bpxpq8X9o6xEHfC_zmoA

Successfully saved authorization token.


In [4]:
import geemap
import datetime
import numpy as np
import math

In [5]:
basin = ee.FeatureCollection('projects/ee-gaoliaoran2020/assets/zhangZT/WeiGan_Basin')
four = ee.FeatureCollection('projects/ee-gaoliaoran2020/assets/zhangZT/2020_the_second_method/four')

In [6]:
def maskS2clouds(image):
    qa = image.select('QA60');
    cloudBitMask = 1 << 10;
    cirrusBitMask = 1 << 11;
    mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
      .And(qa.bitwiseAnd(cirrusBitMask).eq(0));
    return image.updateMask(mask);

In [7]:
# 构建NDVI时间序列
def NDVI_timeMax(start_time,end_time,day_gap,image_number,area):

  def NDVI(image):
    nir = image.select("B8");
    red = image.select("B4");
    ndvi = nir.subtract(red).divide(nir.add(red));
    return image.select().addBands(ndvi);

  image_list = []

  for i in range(image_number.getInfo()):
      s_date = datetime.datetime.strptime(start_time, "%Y-%m-%d") + datetime.timedelta(days = (i)*day_gap)
      e_date = datetime.datetime.strptime(start_time, "%Y-%m-%d") + datetime.timedelta(days = (i+1)*day_gap)
      s_time = s_date.strftime("%Y-%m-%d");
      e_time = e_date.strftime("%Y-%m-%d");
      NDVI_image = ee.ImageCollection("COPERNICUS/S2_SR")\
                    .filterDate(s_time, e_time)\
                    .filterBounds(area).map(NDVI).max().clip(area).rename("NDVI").set("data",s_date)\
                    .set('system:footprint',area.geometry())
      ddiff = ee.Date(s_time).difference(ee.Date(start_time), 'day')
      new_NDVI_image = NDVI_image.addBands(ee.Image(1).toFloat().rename('constant'))\
                                .addBands(ee.Image(ddiff).toFloat().rename('t'))\
                                .addBands(ee.Image(ddiff).pow(ee.Image(2)).toFloat().rename('t2'))\
                                .addBands(ee.Image(ddiff).pow(ee.Image(3)).toFloat().rename('t3'))
      globals()["NDVI_image"+s_date.strftime("%Y%m%d")] = NDVI_image
      image_list.append(globals()["NDVI_image"+s_date.strftime("%Y%m%d")])

  NDVI_col = ee.ImageCollection(image_list)
  NDVI_image = NDVI_col.toBands()
  return NDVI_col,NDVI_image

# 构建NDVI变化时间序列
def NDVI_difference(NDVI_SG_series_col):
  image_list = []
  NDVI_difference_list = []
  NDVI_list = NDVI_SG_series_col.toList(30)
  list_number = len(NDVI_list.getInfo())
  for i in range(list_number-1):
    image_1 = ee.Image(NDVI_list.get(i))
    image_2 = ee.Image(NDVI_list.get(i+1))
    image_difference = image_2.subtract(image_1)  
    globals()["NDVI_difference"+str(i)] = image_difference
    image_list.append(globals()["NDVI_difference"+str(i)])
    NDVI_difference_list.append("NDVI_difference"+'{:0>2d}'.format(i))
  difference_col = ee.ImageCollection(image_list)
  NDVI_differnce_image = difference_col.toBands().rename(NDVI_difference_list)
  return NDVI_differnce_image


# 构建棉花
def BWI_max(start_time,end_time,day_gap,image_number,area):

  def Sentinal_BWI(image):
    image1 = image.select(['B2','B3','B4','B5','B6','B7','B8','B8A','B11','B12']);
    arrayImage1D = image1.toArray();
    arrayImage2D = arrayImage1D.toArray(1);
    coefficientsSentinel = ee.Array([[1.07, -0.68, -0.24, 0.17, -0.04, -0.39, 0.04, 0.36, -0.01, -0.04]]);
    componentsImage = ee.Image(coefficientsSentinel)\
          .matrixMultiply(arrayImage2D).divide(10000)\
          .arrayProject([0])\
          .arrayFlatten([['BWI']]);
      
    BWI =componentsImage.select("BWI").rename('BWI');
    return image.select().addBands(BWI);

  image_list = []
  for i in range(image_number.getInfo()):
      s_date = datetime.datetime.strptime(start_time, "%Y-%m-%d") + datetime.timedelta(days = (i)*day_gap)
      e_date = datetime.datetime.strptime(start_time, "%Y-%m-%d") + datetime.timedelta(days = (i+1)*day_gap)
      s_time = s_date.strftime("%Y-%m-%d");
      e_time = e_date.strftime("%Y-%m-%d");
      BWI_image = ee.ImageCollection("COPERNICUS/S2_SR")\
                    .filterDate(s_time, e_time).map(maskS2clouds)\
                    .filterBounds(area).map(Sentinal_BWI).max().clip(area).rename("BWI").set("data",s_date)\
                    .set('system:footprint',area.geometry())
      ddiff = ee.Date(s_time).difference(ee.Date(start_time), 'day')
      new_BWI_image = BWI_image
      globals()["BWI_image"+s_date.strftime("%Y%m%d")] = new_BWI_image
      image_list.append(globals()["BWI_image"+s_date.strftime("%Y%m%d")])

  col = ee.ImageCollection(image_list)
  BWI_image = col.max().rename("BWI")
  return BWI_image

# 构建SAR时间序列
def SAR(start_time,end_time,day_gap,image_number,area):
  def mask_edge(image):
    edge = image.lt(-30.0);
    maskedImage = image.mask().And(edge.Not());
    return image.updateMask(maskedImage);

  SAR_list = []
  image_list = []
  for i in range(image_number.getInfo()):
      s_date = datetime.datetime.strptime(start_time, "%Y-%m-%d") + datetime.timedelta(days = (i)*day_gap)
      e_date = datetime.datetime.strptime(start_time, "%Y-%m-%d") + datetime.timedelta(days = (i+1)*day_gap)
      s_time = s_date.strftime("%Y-%m-%d");
      e_time = e_date.strftime("%Y-%m-%d");
      SAR_image = ee.ImageCollection('COPERNICUS/S1_GRD')\
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))\
            .filter(ee.Filter.eq('instrumentMode', 'IW'))\
            .select('VV')\
            .map(mask_edge)\
            .filterDate(s_time, e_time)\
            .mean().clip(area).set("data",s_date).set('system:footprint',area.geometry())
      new_SAR_image = SAR_image
      globals()["SAR_image"+s_date.strftime("%Y%m%d")] = new_SAR_image
      image_list.append(globals()["SAR_image"+s_date.strftime("%Y%m%d")])
      SAR_list.append("SAR"+'{:0>2d}'.format(i))

  col = ee.ImageCollection(image_list)
  SAR_image = col.toBands().rename(SAR_list)
  return SAR_image

# 构建GREEN时间序列
def GREEN(start_time,end_time,day_gap,image_number,area):
  def Sentinal_Green(image):
    green = image.select('B3');
    return image.select().addBands(green);

  image_list = []
  Greem_list=[]
  for i in range(image_number.getInfo()):
      s_date = datetime.datetime.strptime(start_time, "%Y-%m-%d") + datetime.timedelta(days = (i)*day_gap)
      e_date = datetime.datetime.strptime(start_time, "%Y-%m-%d") + datetime.timedelta(days = (i+1)*day_gap)
      s_time = s_date.strftime("%Y-%m-%d");
      e_time = e_date.strftime("%Y-%m-%d");
      Green_image = ee.ImageCollection("COPERNICUS/S2_SR")\
                    .filterDate(s_time, e_time)\
                    .filterBounds(area).map(Sentinal_Green).max().clip(area).rename("Green").set("data",s_date)\
                    .set('system:footprint',area.geometry())
      ddiff = ee.Date(s_time).difference(ee.Date(start_time), 'day')
      new_Green_image = Green_image
      globals()["Green_image"+s_date.strftime("%Y%m%d")] = new_Green_image
      image_list.append(globals()["Green_image"+s_date.strftime("%Y%m%d")])
      Greem_list.append("Green"+'{:0>2d}'.format(i))

  col = ee.ImageCollection(image_list)
  Green_image = col.toBands().rename(Greem_list)
  return Green_image

# 构建红边1时间序列
def Rededge1(start_time,end_time,day_gap,image_number,area):

  def Sentinal_RedEdge1(image):
    RedEdge1 = image.select('B5');
    return image.select().addBands(RedEdge1);

  image_list = []
  RedEdge1_list=[]
  for i in range(image_number.getInfo()):
      s_date = datetime.datetime.strptime(start_time, "%Y-%m-%d") + datetime.timedelta(days = (i)*day_gap)
      e_date = datetime.datetime.strptime(start_time, "%Y-%m-%d") + datetime.timedelta(days = (i+1)*day_gap)
      s_time = s_date.strftime("%Y-%m-%d");
      e_time = e_date.strftime("%Y-%m-%d");
      RedEdge1_image = ee.ImageCollection("COPERNICUS/S2_SR")\
                    .filterDate(s_time, e_time)\
                    .filterBounds(area).map(Sentinal_RedEdge1).max().clip(area).rename("RedEdge1").set("data",s_date)\
                    .set('system:footprint',area.geometry())
      ddiff = ee.Date(s_time).difference(ee.Date(start_time), 'day')
      new_RedEdge1_image = RedEdge1_image
      globals()["RedEdge1_image"+s_date.strftime("%Y%m%d")] = new_RedEdge1_image
      image_list.append(globals()["RedEdge1_image"+s_date.strftime("%Y%m%d")])
      RedEdge1_list.append("RedEdge1"+'{:0>2d}'.format(i))

  col = ee.ImageCollection(image_list)
  RedEdge1_image = col.toBands().rename(RedEdge1_list)
  return RedEdge1_image


# 构建红边2时间序列
def Rededge2(start_time,end_time,day_gap,image_number,area):

  def Sentinal_RedEdge2(image):
    RedEdge2 = image.select('B6');
    return image.select().addBands(RedEdge2);

  image_list = []
  RedEdge2_list=[]
  for i in range(image_number.getInfo()):
      s_date = datetime.datetime.strptime(start_time, "%Y-%m-%d") + datetime.timedelta(days = (i)*day_gap)
      e_date = datetime.datetime.strptime(start_time, "%Y-%m-%d") + datetime.timedelta(days = (i+1)*day_gap)
      s_time = s_date.strftime("%Y-%m-%d");
      e_time = e_date.strftime("%Y-%m-%d");
      RedEdge2_image = ee.ImageCollection("COPERNICUS/S2_SR")\
                    .filterDate(s_time, e_time)\
                    .filterBounds(area).map(Sentinal_RedEdge2).max().clip(area).rename("RedEdge2").set("data",s_date)\
                    .set('system:footprint',area.geometry())
      ddiff = ee.Date(s_time).difference(ee.Date(start_time), 'day')
      new_RedEdge2_image = RedEdge2_image
      globals()["RedEdge2_image"+s_date.strftime("%Y%m%d")] = new_RedEdge2_image
      image_list.append(globals()["RedEdge2_image"+s_date.strftime("%Y%m%d")])
      RedEdge2_list.append("RedEdge2"+'{:0>2d}'.format(i))

  col = ee.ImageCollection(image_list)
  RedEdge2_image = col.toBands().rename(RedEdge2_list)
  return RedEdge2_image

## RF分类结果

In [8]:
from math import nan
def corn_singal_add(feature):
    return feature.set("landcover","corn_singal").set("index",0);

def corn_wheat_add(feature):
    return feature.set("landcover","corn_wheat").set("index",1);

def cotton_add(feature):
    return feature.set("landcover","cotton").set("index",2);

def wheat_add(feature):
    return feature.set("landcover","wheat").set("index",3);
    
def fruit_add(feature):
    return feature.set("landcover","fruit").set("index",4);

corn_singal = ee.FeatureCollection('projects/ee-gaoliaoran2020/assets/zhangZT/2020point/corn_singal_pure').map(corn_singal_add).filterBounds(basin)
corn_wheat = ee.FeatureCollection('projects/ee-gaoliaoran2020/assets/zhangZT/2020point/corn_wheat_pure').map(corn_wheat_add).filterBounds(basin)
cotton = ee.FeatureCollection('projects/ee-gaoliaoran2020/assets/zhangZT/2020point/cotton_pure').map(cotton_add).filterBounds(basin)
wheat = ee.FeatureCollection('projects/ee-gaoliaoran2020/assets/zhangZT/2020point/wheat_pure').map(wheat_add).filterBounds(basin)
fruit = ee.FeatureCollection('projects/ee-gaoliaoran2020/assets/zhangZT/2020point/fruit_pure').map(fruit_add).filterBounds(basin)

# 选择样本
Corn_singal_data = corn_singal.randomColumn('random',seed = 2);
Corn_wheat_data = corn_wheat.randomColumn('random',seed = 2);
Cotton_data = cotton.randomColumn('random',seed = 2);
Fruit_data = fruit.randomColumn('random',seed = 2);
Wheat_data = wheat.randomColumn('random',seed = 2);

# 选择选择训练样本 
Corn_singal_train = Corn_singal_data.filter(ee.Filter.lt('random', 0.7));
Corn_wheat_train = Corn_wheat_data.filter(ee.Filter.lt('random', 0.7));
Cotton_train = Cotton_data.filter(ee.Filter.lt('random', 0.7));
Fruit_train = Fruit_data.filter(ee.Filter.lt('random', 0.7));
Wheat_train = Wheat_data.filter(ee.Filter.lt('random', 0.7));

TRAIN = ee.FeatureCollection(Corn_singal_train).merge(Corn_wheat_train).merge(Cotton_train).merge(Wheat_train).merge(Fruit_train);

    
# 选择验证样本
Corn_singal_test = Corn_singal_data.filter(ee.Filter.gte('random', 0.7));
Corn_wheat_test = Corn_wheat_data.filter(ee.Filter.gte('random', 0.7));
Cotton_test = Cotton_data.filter(ee.Filter.gte('random', 0.7));
Fruit_test = Fruit_data.filter(ee.Filter.gte('random', 0.7));
Wheat_test = Wheat_data.filter(ee.Filter.gte('random', 0.7));


TEST = ee.FeatureCollection(Corn_singal_test).merge(Corn_wheat_test).merge(Cotton_test).merge(Wheat_test).merge(Fruit_test);

def classification(image, out, numberOfTrees, variablesPerSplit, minLeafPopulation, bagFraction, maxNodes, seed):
  TRAIN_LABEL = image.sampleRegions(TRAIN, scale=10, tileScale=8)
  TEST_LABEL = image.sampleRegions(TEST, scale=10, tileScale=8)
  label = "index"
  bands = image.bandNames()

  trained = ee.Classifier.smileRandomForest(
      numberOfTrees=numberOfTrees,
      variablesPerSplit=variablesPerSplit, 
      minLeafPopulation=minLeafPopulation, 
      bagFraction=bagFraction, 
      maxNodes=maxNodes, 
      seed=seed
  ).train(TRAIN_LABEL, label, bands)

  test = TEST_LABEL.classify(trained);

  confusionMatrix = test.errorMatrix('index', 'classification');
  Matrix = np.array(confusionMatrix.getInfo())
  F1_score = []
  for i in range(len(Matrix)):
    X = sum(Matrix[i])
    x = sum(Matrix[:,i])
    TP = Matrix[i,i]
    P = TP/x
    R = TP/X
    F1 = 2*P*R/(P+R)
    if np.isnan(F1):
      F1_score.append(0);
    else:
      F1_score.append(F1)
  print('Confusion Matrix', confusionMatrix.getInfo())
  print('overall accuracy', confusionMatrix.accuracy().getInfo());
  # 面板上显示总体精度
  print('kappa accuracy', confusionMatrix.kappa().getInfo());
  # 面板上显示kappa值
  print('F1 score', F1_score)
  # result_probility = F1_score
  result_calssification = out.classify(trained)
  trained = trained.setOutputMode("MULTIPROBABILITY")
  result_probility = out.classify(trained).arrayFlatten([['class1', 'class2', 'class3', 'class4', 'class5']])
  return result_calssification,result_probility

In [9]:
# 设置时间序列开始和时间序列结束的时间点
start_time = "2020-02-01"
end_time = "2020-12-31"
day_gap = 16
start = ee.Date(start_time);
end = ee.Date(end_time);
day_number = end.difference(start, "day").floor()
image_number = day_number.divide(day_gap).floor()
print(image_number.getInfo())


NDVI = NDVI_timeMax(start_time,end_time,day_gap,image_number,four)
NDVI_col = NDVI[0]
NDVI_image = NDVI[1]
NDVI_differnce_image = NDVI_difference(NDVI_col)
BWI_image = BWI_max(start_time,end_time,day_gap,image_number,four)
SAR_image = SAR(start_time,end_time,day_gap,image_number,four)
image = ee.Image(NDVI_differnce_image).addBands(NDVI_image).addBands(BWI_image).addBands(SAR_image)#.addBands(Green_image)#.addBands(RedEdge1_image).addBands(RedEdge2_image)


result_all_series = classification(image,image,500,2,1,0.85,20,0)
classification_all_series = result_all_series[0]
# accuracy_all_series = result_all_series[1]
accuracy_all_series = result_all_series[1].reduce(reducer=ee.Reducer.max()).rename("max")

20
Confusion Matrix [[21, 0, 0, 0, 0], [0, 14, 0, 0, 2], [0, 0, 21, 0, 0], [0, 0, 0, 4, 0], [2, 0, 0, 0, 16]]
overall accuracy 0.95
kappa accuracy 0.9348534201954397
F1 score [0.9545454545454545, 0.9333333333333333, 1.0, 1.0, 0.8888888888888888]


## SVM分类结果

In [ ]:
def classification(image, out):
  TRAIN_LABEL = image.sampleRegions(TRAIN, scale=10, tileScale=8)
  TEST_LABEL = image.sampleRegions(TEST, scale=10, tileScale=8)
  label = "index"
  bands = image.bandNames()

  trained = ee.Classifier.libsvm().train(TRAIN_LABEL, label, bands)

  test = TEST_LABEL.classify(trained);

  confusionMatrix = test.errorMatrix('index', 'classification');
  Matrix = np.array(confusionMatrix.getInfo())
  F1_score = []
  for i in range(len(Matrix)):
    X = sum(Matrix[i])
    x = sum(Matrix[:,i])
    TP = Matrix[i,i]
    P = TP/x
    R = TP/X
    F1 = 2*P*R/(P+R)
    if np.isnan(F1):
      F1_score.append(0);
    else:
      F1_score.append(F1)
  print('Confusion Matrix', confusionMatrix.getInfo())
  print('overall accuracy', confusionMatrix.accuracy().getInfo());
  # 面板上显示总体精度
  print('kappa accuracy', confusionMatrix.kappa().getInfo());
  # 面板上显示kappa值
  print('F1 score', F1_score)
  # result_probility = F1_score
  result_calssification = out.classify(trained)
  return result_calssification

result_all_series = classification(image,image)
classification_all_series = result_all_series

Confusion Matrix [[19, 0, 0, 1, 1], [0, 8, 1, 3, 4], [5, 0, 16, 0, 0], [1, 1, 0, 2, 0], [3, 0, 0, 0, 15]]
overall accuracy 0.75
kappa accuracy 0.6752587781611528
F1 score [0.7755102040816326, 0.64, 0.8421052631578947, 0.4, 0.7894736842105262]


## GradientTreeBoost

In [ ]:
def classification(image, out):
  TRAIN_LABEL = image.sampleRegions(TRAIN, scale=10, tileScale=8)
  TEST_LABEL = image.sampleRegions(TEST, scale=10, tileScale=8)
  label = "index"
  bands = image.bandNames()

  trained = ee.Classifier.smileGradientTreeBoost(500).train(TRAIN_LABEL, label, bands)

  test = TEST_LABEL.classify(trained);

  confusionMatrix = test.errorMatrix('index', 'classification');
  Matrix = np.array(confusionMatrix.getInfo())
  F1_score = []
  for i in range(len(Matrix)):
    X = sum(Matrix[i])
    x = sum(Matrix[:,i])
    TP = Matrix[i,i]
    P = TP/x
    R = TP/X
    F1 = 2*P*R/(P+R)
    if np.isnan(F1):
      F1_score.append(0);
    else:
      F1_score.append(F1)
  print('Confusion Matrix', confusionMatrix.getInfo())
  print('overall accuracy', confusionMatrix.accuracy().getInfo());
  # 面板上显示总体精度
  print('kappa accuracy', confusionMatrix.kappa().getInfo());
  # # 面板上显示kappa值
  print('F1 score', F1_score)
  # result_probility = F1_score
  result_calssification = out.classify(trained)
  return result_calssification

result_all_series = classification(image,image)
classification_all_series = result_all_series

Confusion Matrix [[21, 0, 0, 0, 0], [0, 13, 0, 1, 2], [0, 0, 20, 0, 1], [0, 0, 0, 4, 0], [1, 2, 0, 0, 15]]
overall accuracy 0.9125
kappa accuracy 0.8865018240778272
F1 score [0.9767441860465117, 0.8387096774193549, 0.975609756097561, 0.888888888888889, 0.8333333333333334]


In [ ]:
geemap.ee_export_image_to_asset(classification_all_series.toInt(), 
                         description='GradientTreeBoost', 
                         assetId="projects/ee-gaoliaoran2020/assets/GradientTreeBoost", 
                         dimensions=None, 
                         region=four.geometry(), 
                         scale=10, 
                         crs="EPSG:4326", 
                         maxPixels=1e13)

## CART

In [ ]:
def classification(image, out):
  TRAIN_LABEL = image.sampleRegions(TRAIN, scale=10, tileScale=8)
  TEST_LABEL = image.sampleRegions(TEST, scale=10, tileScale=8)
  label = "index"
  bands = image.bandNames()

  trained = ee.Classifier.smileCart(500,4).train(TRAIN_LABEL, label, bands)

  test = TEST_LABEL.classify(trained);

  confusionMatrix = test.errorMatrix('index', 'classification');
  Matrix = np.array(confusionMatrix.getInfo())
  F1_score = []
  for i in range(len(Matrix)):
    X = sum(Matrix[i])
    x = sum(Matrix[:,i])
    TP = Matrix[i,i]
    P = TP/x
    R = TP/X
    F1 = 2*P*R/(P+R)
    if np.isnan(F1):
      F1_score.append(0);
    else:
      F1_score.append(F1)
  print('Confusion Matrix', confusionMatrix.getInfo())
  print('overall accuracy', confusionMatrix.accuracy().getInfo());
  # 面板上显示总体精度
  print('kappa accuracy', confusionMatrix.kappa().getInfo());
  # # 面板上显示kappa值
  print('F1 score', F1_score)
  # result_probility = F1_score
  result_calssification = out.classify(trained)
  return result_calssification

result_all_series = classification(image,image)
classification_all_series = result_all_series

Confusion Matrix [[21, 0, 0, 0, 0], [0, 14, 0, 0, 2], [2, 1, 17, 0, 1], [0, 0, 0, 4, 0], [1, 2, 0, 0, 15]]
overall accuracy 0.8875
kappa accuracy 0.8538664501725187
F1 score [0.9333333333333333, 0.8484848484848485, 0.8947368421052632, 1.0, 0.8333333333333334]


In [ ]:
geemap.ee_export_image_to_asset(classification_all_series.toInt(), 
                         description='CART', 
                         assetId="projects/ee-gaoliaoran2020/assets/CART", 
                         dimensions=None, 
                         region=four.geometry(), 
                         scale=10, 
                         crs="EPSG:4326", 
                         maxPixels=1e13)

In [ ]:
geemap.ee_export_image_to_drive( 
        image=image.toFloat(), 
        description="image",
        region=four.geometry(),
        scale=500, 
        crs="EPSG:4326", 
        maxPixels=1e13 
      )

In [ ]:
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [ ]:
Map.addLayer(right_rate)
#              ,{
#   "min": 0, 
#   "max": 5,
#   "palette": ["#ffffff","267000","ffd300","ff2626","a57000","93cc93"]
# })

In [ ]:
from math import nan
def corn_singal_add(feature):
    return feature.set("landcover","corn_singal").set("index",1);

def corn_wheat_add(feature):
    return feature.set("landcover","corn_wheat").set("index",2);

def cotton_add(feature):
    return feature.set("landcover","cotton").set("index",3);

def wheat_add(feature):
    return feature.set("landcover","wheat").set("index",4);
    
def fruit_add(feature):
    return feature.set("landcover","fruit").set("index",5);

corn_singal = ee.FeatureCollection('projects/ee-gaoliaoran2020/assets/zhangZT/2020point/corn_singal_pure').map(corn_singal_add).filterBounds(basin)
corn_wheat = ee.FeatureCollection('projects/ee-gaoliaoran2020/assets/zhangZT/2020point/corn_wheat_pure').map(corn_wheat_add).filterBounds(basin)
cotton = ee.FeatureCollection('projects/ee-gaoliaoran2020/assets/zhangZT/2020point/cotton_pure').map(cotton_add).filterBounds(basin)
wheat = ee.FeatureCollection('projects/ee-gaoliaoran2020/assets/zhangZT/2020point/wheat_pure').map(wheat_add).filterBounds(basin)
fruit = ee.FeatureCollection('projects/ee-gaoliaoran2020/assets/zhangZT/2020point/fruit_pure').map(fruit_add).filterBounds(basin)

# 选择样本
Corn_singal_data = corn_singal.randomColumn('random',seed = 2);
Corn_wheat_data = corn_wheat.randomColumn('random',seed = 2);
Cotton_data = cotton.randomColumn('random',seed = 2);
Fruit_data = fruit.randomColumn('random',seed = 2);
Wheat_data = wheat.randomColumn('random',seed = 2);

# 选择选择训练样本 
Corn_singal_train = Corn_singal_data.filter(ee.Filter.lt('random', 0.7));
Corn_wheat_train = Corn_wheat_data.filter(ee.Filter.lt('random', 0.7));
Cotton_train = Cotton_data.filter(ee.Filter.lt('random', 0.7));
Fruit_train = Fruit_data.filter(ee.Filter.lt('random', 0.7));
Wheat_train = Wheat_data.filter(ee.Filter.lt('random', 0.7));

ValidationTRAIN = ee.FeatureCollection(Corn_singal_train).merge(Corn_wheat_train).merge(Cotton_train).merge(Wheat_train).merge(Fruit_train);
   
# 选择验证样本
Corn_singal_test = Corn_singal_data.filter(ee.Filter.gte('random', 0.7));
Corn_wheat_test = Corn_wheat_data.filter(ee.Filter.gte('random', 0.7));
Cotton_test = Cotton_data.filter(ee.Filter.gte('random', 0.7));
Fruit_test = Fruit_data.filter(ee.Filter.gte('random', 0.7));
Wheat_test = Wheat_data.filter(ee.Filter.gte('random', 0.7));

ValidationTEST = ee.FeatureCollection(Corn_singal_test).merge(Corn_wheat_test).merge(Cotton_test).merge(Wheat_test).merge(Fruit_test);

Validation = classification.sampleRegions(ValidationTEST, scale=10, tileScale=8)
confusionMatrix = Validation.errorMatrix('index', 'sum');
Matrix = np.array(confusionMatrix.getInfo())
F1_score = []
for i in range(len(Matrix)):
  X = sum(Matrix[i])
  x = sum(Matrix[:,i])
  TP = Matrix[i,i]
  P = TP/x
  R = TP/X
  F1 = 2*P*R/(P+R)
  if np.isnan(F1):
    F1_score.append(0);
  else:
    F1_score.append(F1)
print('Confusion Matrix', confusionMatrix.getInfo())
print('overall accuracy', confusionMatrix.accuracy().getInfo());
# 面板上显示总体精度
print('kappa accuracy', confusionMatrix.kappa().getInfo());
# 面板上显示kappa值
print('F1 score', F1_score)

AttributeError: ignored